In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
# Load the datasets
db_data_path = '/content/drive/MyDrive/Assessment/DB_Data.csv'
ehr_data_path = '/content/drive/MyDrive/Assessment/EHR_Data.csv'



In [4]:
db_data = pd.read_csv('/content/drive/MyDrive/Assessment/DB_Data.csv')
ehr_data = pd.read_csv('/content/drive/MyDrive/Assessment/EHR_Data.csv')

# Display the first few rows of each dataset to understand their structure
db_data.head(), ehr_data.head()



(  Patient Name       Provider Name from_date_range                  cpt_codes
 0  Lucas Smith          Aiden King      2024-07-03  {97110,97112,97140,97010}
 1  Lucas Smith  Charlotte Williams      2024-07-11  {97112,97010,97140,97110}
 2  Lucas Smith  Charlotte Williams      2024-07-17  {97110,97140,97112,97010}
 3  Lucas Smith          Julian Lee      2024-07-27  {97140,97110,97112,97010}
 4  Lucas Smith          Aiden King      2024-08-07  {97010,97110,97112,97140},
    Patient_Name       Provider_Name Date_of_Service CPT_Code
 0   Ava Johnson          Liam Young       9/24/2024    97010
 1   Ava Johnson          Liam Young       9/24/2024    97140
 2   Sophia King    Sebastian Miller        7/3/2024    97140
 3  Carter Brown  Charlotte Williams        9/9/2024    97014
 4  Olivia Davis            Noah Lee        9/9/2024    97140)

In [5]:
# Standardize column names for consistency
db_data.rename(columns={
    "Patient Name": "Patient_Name",
    "Provider Name": "Provider_Name",
    "from_date_range": "Date_of_Service",
    "cpt_codes": "CPT_Codes"
}, inplace=True)



In [6]:
# Ensure date formats are consistent
db_data["Date_of_Service"] = pd.to_datetime(db_data["Date_of_Service"])
ehr_data["Date_of_Service"] = pd.to_datetime(ehr_data["Date_of_Service"], errors='coerce')

In [7]:
# Create unique identifiers for each encounter
db_data["Unique_ID"] = db_data["Patient_Name"] + "|" + db_data["Provider_Name"] + "|" + db_data["Date_of_Service"].astype(str)
ehr_data["Unique_ID"] = ehr_data["Patient_Name"] + "|" + ehr_data["Provider_Name"] + "|" + ehr_data["Date_of_Service"].astype(str)

In [8]:
# Aggregate EHR data to create a list of CPT codes per unique encounter
ehr_grouped = ehr_data.groupby("Unique_ID").agg({
    "CPT_Code": lambda x: set(x),
    "Patient_Name": "first",
    "Provider_Name": "first",
    "Date_of_Service": "first"
}).reset_index()

ehr_grouped.rename(columns={"CPT_Code": "CPT_Codes"}, inplace=True)

In [9]:
# Display the preprocessed data for verification
db_data.head(), ehr_grouped.head()

(  Patient_Name       Provider_Name Date_of_Service                  CPT_Codes  \
 0  Lucas Smith          Aiden King      2024-07-03  {97110,97112,97140,97010}   
 1  Lucas Smith  Charlotte Williams      2024-07-11  {97112,97010,97140,97110}   
 2  Lucas Smith  Charlotte Williams      2024-07-17  {97110,97140,97112,97010}   
 3  Lucas Smith          Julian Lee      2024-07-27  {97140,97110,97112,97010}   
 4  Lucas Smith          Aiden King      2024-08-07  {97010,97110,97112,97140}   
 
                                    Unique_ID  
 0          Lucas Smith|Aiden King|2024-07-03  
 1  Lucas Smith|Charlotte Williams|2024-07-11  
 2  Lucas Smith|Charlotte Williams|2024-07-17  
 3          Lucas Smith|Julian Lee|2024-07-27  
 4          Lucas Smith|Aiden King|2024-08-07  ,
                            Unique_ID CPT_Codes Patient_Name Provider_Name  \
 0  Aiden Allen|Aiden King|2024-07-03   {97113}  Aiden Allen    Aiden King   
 1  Aiden Allen|Aiden King|2024-07-10   {97113}  Aiden Allen 

In [10]:
# Identify missing encounters: EHR Unique IDs not in DB Unique IDs
missing_ids = ehr_grouped[~ehr_grouped["Unique_ID"].isin(db_data["Unique_ID"])]

In [11]:
# Extract details of missing encounters
missing_encounters = missing_ids[["Patient_Name", "Provider_Name", "Date_of_Service", "CPT_Codes"]]

In [13]:
# Display the missing encounters
missing_encounters.reset_index(drop=True, inplace=True)
missing_encounters, missing_encounters.shape

(    Patient_Name       Provider_Name Date_of_Service  \
 0    Aiden Allen          Aiden King      2024-08-28   
 1    Aiden Allen          Aiden King      2024-09-04   
 2    Aiden Allen          Aiden King      2024-09-10   
 3    Aiden Allen          Aiden King      2024-09-25   
 4    Aiden Allen          Liam Young      2024-09-25   
 ..           ...                 ...             ...   
 277  Zoey Martin          Aiden King      2024-07-29   
 278  Zoey Martin          Aiden King      2024-08-15   
 279  Zoey Martin          Aiden King      2024-09-30   
 280   Zoey Perez  Charlotte Williams      2024-08-14   
 281   Zoey Perez  Charlotte Williams      2024-08-21   
 
                               CPT_Codes  
 0                        {97113, NORCM}  
 1                        {97113, NORCM}  
 2                 {97113, NORCM, 97010}  
 3                        {97113, NORCM}  
 4                 {97140, 97110, 97010}  
 ..                                  ...  
 277  {sp90, 